Try to implement the paper Hybrid Bayesian Neural Networks

In [7]:
# IMPORTS
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp

Create training and evaluation datasets

In [8]:

def get_train_and_test_splits(train_size, batch_size=1):
    # Read the dataset from a CSV file
    data = pd.read_csv('data/wine_quality/wine_quality.csv')
    features = data.drop('quality', axis=1).values
    labels = data['quality'].values

    # Calculate the number of samples for the training set
    n_samples = len(data)
    train_size = int(n_samples * train_size)

    # Randomly shuffle the data
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    shuffled_features = features[indices]
    shuffled_labels = labels[indices]

    # Split the dataset into training and testing sets
    X_train = shuffled_features[:train_size]
    y_train = shuffled_labels[:train_size]
    X_test = shuffled_features[train_size:]
    y_test = shuffled_labels[train_size:]

    # Convert the data to TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    # Cast the labels to float32 and apply transformations
    train_dataset = train_dataset.map(lambda x, y: (x, tf.cast(y, tf.float32)))
    test_dataset = test_dataset.map(lambda x, y: (x, tf.cast(y, tf.float32)))
    train_dataset = train_dataset.shuffle(buffer_size=train_size).batch(batch_size).cache()
    test_dataset = test_dataset.batch(batch_size)

    return train_dataset, test_dataset

Compile, train, and evaluate the model

In [ ]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

Create model inputs

In [ ]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

Experiment 1: standard neural network

In [ ]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)


num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

In [ ]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")